# Fourier Series

Fourier series are powerful function and signal representation, capable of representing
any periodic bandlimited signal. Understanding the representation can be tricky, but is
greatly helped by visualising individual components of the series, which is what we'll do
here.

Citing <a href="https://en.wikipedia.org/wiki/Fourier_series">Wikipedia</a>: A Fourier
series is a periodic function composed of harmonically related sinusoids comined by a
weighted summation. With appropriate weights, one cycle (or period) of the summation can
be made to approximate as arbitrary function in that interval, or the entire function if
it itself is periodic.

The fourier series representation of a function (Exponential Form) is given by:

$$x_{T}(t)=\sum_{n=-\infty}^{\infty} c_{n} e^{j n \omega_{0} t}$$

Where:
* $\omega_{0}$ is the fundamental frequency
* $n$ is an integer representing the harmonics of $\omega_{0}$
* $c_n$ are the coefficients of the infinite series

In [1]:
import matplotlib.pyplot as plt
import math
import numpy as np
plt.rcParams['axes.labelsize']= 'medium'

## Square Waves

Let's start with an informative example - representing a Square Wave by it's fourier
series, this is an interesting example, as how can smooth sinusoids approximate a function
containing discrete steps?

The coefficients of the fourier series for the square wave are given by the following
expression, each coefficient $c_n$ determining the contribution of a sinusoid at the given
frequency $n\omega_{0}$.

$$c_n = \frac{A}{\pi n} \sin \left(n \omega_{0} \frac{T_{p}}{2}\right)$$

## The Composite Signal

Given the value of $\omega_{0}$ is X, $T_p$ is X and $A$ is Y the first ($N$) will produce a function approximating the square as shown below.


So it turns out that we need quite a lot of components to approximate the square wave well, try setting the current value of $N=$ X to the maximum of 100. The number of components has a significant impact of the final signal and we would need the full infiite series in order to converge to a true square wave with instantaneous value changes.

The parameter $A$ controls amplitude and has no influence on the structure of the approximation, try changing it to see, A: X.

The parameters $\omega_{0}$ (X) and $T_p$ (X) have the greatest influence on the waveform shape, controlling the periodicity, duty cycle and rate convergence of the representation to the target waveform. try adjustuing these in tandem so see how the waveform is affected.

In [6]:
#| label: fig-fourier-playground

import matplotlib.pyplot as plt
import math
import numpy as np
import ipywidgets as ipy
from ipywidgets import VBox, HBox


def square_coefficients(A=1, Tp=0.5, wo=0.5, N=20):
    rwo = wo*2*math.pi
    return [(A/(math.pi*(n+1)))*math.sin((n+1)*rwo*(Tp/2)) for n in range(0,N)]

def series(c, wo, N):
    rwo = wo*2*math.pi
    t = np.arange(-3*math.pi,3*math.pi,0.01)
    X = np.zeros((N, t.shape[0]), dtype=complex)
    for n in range(0,N):
        X[n, :] = c[n]*np.exp(-1j*(n+1)*wo*t)
    return t, X


w_A = ipy.FloatSlider(value=1.0, min=0.5, max=3, step=0.5, description="Amplitude (A):")
w_Tp = ipy.FloatSlider(value=0.5, min=0.1, max=0.9, step=0.1, description="Mark/Space Ratio (Tp):")
w_wo = ipy.FloatSlider(value=0.5, min=0.1, max=2.0, step=0.1, description="Fundamental Frequency (wo):")
w_N = ipy.IntSlider(value=10, min=1, max=100, step=1, description="Num Components (N):")

controls = HBox([VBox([w_A, w_N]), VBox([w_Tp, w_wo])])

def format_axes(ax):
    plt.grid(True, 'major', 'y')
    plt.axis('tight')
    ax.set_frame_on(False)
    ax.set_xticks([])
    ax.set_yticklabels([])
    

def plot_coefficients(A, Tp, wo, N):
    c = square_coefficients(A, Tp, wo, N)
    fig, ax = plt.subplots(1,1, figsize=(12,2))
    plt.bar(range(0,len(c)), c)
    plt.title('coefficient series')
    format_axes(ax)

def plot_composite(A, Tp, wo, N):
    c = square_coefficients(A, Tp, wo, N)
    t, X = series(c, wo, N)
    S = np.sum(X, axis=0);
    fig, ax = plt.subplots(1,1, figsize=(12,2))
    plt.plot(t, np.real(S))
    plt.plot(t, np.imag(S))
    format_axes(ax)
    plt.title('composite signal')

def plot_components(A, Tp, wo, N):
    c = square_coefficients(A, Tp, wo, N)
    t, X = series(c, wo, N)
    S = np.sum(X, axis=0);
    fig, axs = plt.subplots(2,1, figsize=(12,4))
    axs[0].plot(t,np.real(X).T)
    axs[0].set_title('series components')
    axs[0].set_xlabel('real')
    axs[1].plot(t,np.imag(X).T)
    axs[1].set_xlabel('imaginary')
    format_axes(axs[0])
    format_axes(axs[1])
    
    
heading = ipy.HTML(value='<div style="text-align:center;font-size:20px">Fourier Series Playground</div>')
coeff_plot = ipy.interactive_output(plot_coefficients, dict(A=w_A, Tp=w_Tp, wo=w_wo, N=w_N))
composite_plot = ipy.interactive_output(plot_composite, dict(A=w_A, Tp=w_Tp, wo=w_wo, N=w_N))
component_plot = ipy.interactive_output(plot_components, dict(A=w_A, Tp=w_Tp, wo=w_wo, N=w_N))

ui = VBox([heading, controls, coeff_plot, composite_plot, component_plot], layout=ipy.Layout(display="flex", align_items="center"))

display(ui)
